In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
pip install git+https://github.com/theislab/drvi.git@main

In [ ]:
pip install gprofiler

In [ ]:
import warnings

In [ ]:
pip install anndata

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
import anndata as ad
import scanpy as sc
from matplotlib import pyplot as plt
from IPython.display import display
from gprofiler import gprofiler
import torch
import numpy as np
import drvi
from drvi.model import DRVI

In [ ]:
sc.settings.set_figure_params(dpi=100, frameon=False)
sc.set_figure_params(dpi=100)
sc.set_figure_params(figsize=(3, 3))
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.figsize"] = (3, 3)

In [ ]:
adata = sc.read("/work/DRVI/fixed_data/HH/healthy_hamstring_processed_adata_raw_nonormalization.h5ad")

In [ ]:
adata

In [ ]:
adata.layers["counts"] = adata.X.copy()

In [ ]:
# Split the data set into train and test
from sklearn.model_selection import train_test_split


split_key = "split"
adata.obs[split_key] = "train"
idx = list(range(len(adata)))
idx_train, idx_test = train_test_split(adata.obs_names, test_size=0.1, random_state=42)
adata.obs.loc[idx_train, split_key] = "train"
adata.obs.loc[idx_test, split_key] = "test"

In [ ]:
train_adata = adata[adata.obs[split_key] == "train"].copy()
test_adata = adata[adata.obs[split_key] == "test"].copy()

In [ ]:
test_adata.write("test_adata_CHECKING.h5ad")

In [ ]:
# Setup data
DRVI.setup_anndata(
    train_adata,
    # DRVI accepts count data by default.
    # Do not forget to change gene_likelihood if you provide a non-count data.
    layer="counts",
    # Always provide a list. DRVI can accept multiple covariates.
    categorical_covariate_keys=["cell_type", "sex", "donor_id"],
    # DRVI accepts count data by default.
    # Set to false if you provide log-normalized data and use normal distribution (mse loss).
    is_count_data=True,
)

# construct the model
model = DRVI(
    train_adata,
    # Provide categorical covariates keys once again. Refer to advanced usages for more options.
    categorical_covariates=["cell_type", "sex", "donor_id"],
    n_latent=128,
    # For encoder and decoder dims, provide a list of integers.
    encoder_dims=[128, 128],
    decoder_dims=[128, 128],
)
model

In [ ]:
# For cpu training you should add the following line to the model.train parameters:
#accelerator="cpu", devices=1,
#
# For mps acceleration on macbooks, add the following line to the model.train parameters:
# accelerator="mps", devices=1,
#
# For gpu training don't provide any additional parameter.
# More details here: https://lightning.ai/docs/pytorch/stable/accelerators/gpu_basic.html

n_epochs = 400

# train the model
model.train(
    max_epochs=n_epochs,
    early_stopping=True,
    early_stopping_patience=20,
    # mps
    # accelerator="mps", devices=1,
    # cpu
    #accelerator="cpu", devices=1,
    # gpu: no additional parameter
    #
    # No need to provide `plan_kwargs` if n_epochs >= 400.
    plan_kwargs={
        "n_epochs_kl_warmup": n_epochs,
    },
)

# Runtime:
# The runtime for CPU laptop (M1) is 208 minutes
# The runtime for Macbook gpu (M1) is 64 minutes
# The runtime for GPU (A100) is 17 minutes

In [ ]:
model.save("FIXED_trained_models/DRVI_HH_final_covariate_check/", adata = adata)

In [ ]:
model = model.load("/work/DRVI/FIXED_trained_models/DRVI_HH_final_raw", adata = train_adata)

In [ ]:
model

In [ ]:
def predict(model, adata):
    model._validate_anndata(adata)
    model.module.eval()

    scdl = model._make_data_loader(adata=adata, indices=None, batch_size=128, shuffle=False)
    mus = []
    for tensors in scdl:
        inference_outputs, generative_outputs = model.module.forward(
                    tensors,
                    compute_loss=False,
                )
        _mus = torch.nan_to_num(generative_outputs['px'].mean, nan=0, neginf=0, posinf=100) 
        mus.append(_mus.detach().cpu().numpy())
    mus = np.concatenate(mus, axis=0)
    out_adata = adata.copy()
    out_adata.X = mus
    return out_adata


model._validate_anndata(test_adata)
rec = predict(model, test_adata)


In [ ]:
adata_post.write("adata_post_with_reconstruction_DRVI_fixed.h5ad")

In [ ]:
# If rec is an AnnData object, extract the X attribute (i.e., the data matrix)
import anndata
if isinstance(rec, anndata.AnnData):
    rec = rec.X

# Now, rec should be a numpy array or sparse matrix, which is what obsm expects
test_adata.obsm["X_reconstructed"] = rec

# Save the entire object with the reconstructed data
test_adata.write("adata_post_with_latent_and_reconstructed_HH_DRVI_raw_fix.h5ad")


In [ ]:
y_true = test_adata.X

In [ ]:
rec1 = rec.X

In [ ]:
rec.X

In [ ]:
import anndata
# Ensure y_true is dense
y_true = test_adata.X
if not isinstance(y_true, np.ndarray):
    y_true = y_true.toarray()

# Ensure rec is dense
if isinstance(rec, anndata.AnnData):  # Only access .X if rec is an AnnData object
    rec = rec.X

if not isinstance(rec, np.ndarray):   # Convert to dense if it's still sparse
    rec = rec.toarray()


In [ ]:

# Flatten for R²
y_true_flat = y_true.flatten()
y_pred_flat = rec1.flatten()

In [ ]:
latent = model.get_latent_representation(adata)

In [ ]:
# Compute R²
r2 = r2_score(y_true_flat, y_pred_flat)
print(f"R² Score: {r2}")

In [ ]:
mae = mean_absolute_error(y_true_flat, y_pred_flat)
print("MAE:", mae)

In [ ]:
mse = mean_squared_error(y_true_flat, y_pred_flat)
print("MSE:", mse)

In [ ]:
sc.read("adata_post_with_reconstruction_DRVI.h5ad")

In [ ]:
adata_post = test_adata.copy()
adata_post.obsm["X_reconstructed_DRVI"] = rec.X 


In [ ]:
print(dir(model))  # Replace `model` with your DRVI model variab'
model.get_latent_representation

In [ ]:
z = model.get_latent_representation(
    adata,
    batch_size=256,)

In [ ]:
import scanpy as sc
import pandas as pd

latent_adata = sc.AnnData(z)
latent_adata.obs = train_adata.obs.copy()


In [ ]:
sc.pp.neighbors(latent_adata, use_rep="X")  # Use latent space for neighbor graph
sc.tl.umap(latent_adata)  # Run UMAP


In [ ]:
sc.pl.umap(latent_adata, color=["cell_type"])


In [ ]:
sc.pp.neighbors(z, n_neighbors=10, use_rep="X", n_pcs=embed.X.shape[1])
sc.tl.umap(z, spread=1.0, min_dist=0.5, random_state=123)
sc.pp.pca(z)

In [ ]:
sc.pl.umap(z, color=["cell_type"], ncols=1, frameon=False)

In [ ]:
z_basal = model.get_latent_representation(
    adata,
    batch_size=256,)

In [ ]:
embed = ad.AnnData(model.get_latent_representation(), obs=train_adata.obs)
sc.pp.subsample(embed, fraction=1.0)  # Shuffling for better visualization of overlapping colors

sc.pp.neighbors(embed, n_neighbors=10, use_rep="X", n_pcs=embed.X.shape[1])
sc.tl.umap(embed, spread=1.0, min_dist=0.5, random_state=123)
sc.pp.pca(embed)

embed.write("drvi_HH_128_embed.h5ad")

In [ ]:
embed = sc.read("drvi_HH_128_embed.h5ad")

In [ ]:
sc.pl.umap(embed, color=["cell_type"], ncols=1, frameon=False)

In [ ]:
test_adata

In [ ]:
adata

In [ ]:
print(f"Shape of y_true (filtered adata): {adata_2.shape}")  # Should be (1002, 22692)
print(f"Shape of y_pred (rec): {rec_2.shape}")  # Should be (1002, 22692)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.cluster import KMeans
from sklearn.metrics import mutual_info_score
from sklearn.model_selection import train_test_split
###MIG###
import numpy as np
import anndata
import pandas as pd

def encode_categorical(data):
    encoders = []
    encoded_data = np.zeros_like(data, dtype=int)
    for i in range(data.shape[1]):
        le = LabelEncoder()
        encoded_data[:, i] = le.fit_transform(data[:, i])
        encoders.append(le)
    return encoded_data, encoders

def prep_data(adata, embedding, covriate_keys=None):
    encoded_factors_of_variation, _ = encode_categorical(adata.obs[covriate_keys].values)

    if isinstance(embedding, anndata.AnnData):  
        embedding_data = embedding.X
    else:
        embedding_data = embedding

    mus = np.array(embedding_data)
    ys = np.array(encoded_factors_of_variation)

    return mus.T.copy(), ys.T.copy()


def compute_mig(mus, ys, covariate_names=None):
    """Computes the mutual information gap."""
    return _compute_mig(mus, ys, covariate_names)

def _compute_mig(mus, ys, covariate_names=None):
    """Computes MIG score based on latent codes and covariates."""
    score_dict = {}
    discretized_mus = make_discretizer(mus, discretizer_fn=_histogram_discretize)
   # print("Sample Discretized Latent Variables:\n", discretized_mus[:, :5])
    m = discrete_mutual_info(discretized_mus, ys)

    if covariate_names is None:
        covariate_names = [f"Covariate {j}" for j in range(m.shape[1])]
        
    for j in range(m.shape[1]):
        top_indices = np.argsort(m[:, j])[::-1][:3]
        top_scores = m[top_indices, j]
        print(f"Top 3 MI scores for covariate '{covariate_names[j]}':")
        for idx, score in zip(top_indices, top_scores):
            print(f"  Latent dim {idx}: MI = {score:.4f}")

    assert m.shape[0] == mus.shape[0]
    assert m.shape[1] == ys.shape[0]

    entropy = discrete_entropy(ys)
    sorted_m = np.sort(m, axis=0)[::-1]

    score_dict["discrete_mig"] = np.mean(
        np.divide(sorted_m[0, :] - sorted_m[1, :], entropy[:])
    )

    print("MIG score:", score_dict)
    print("Entropy values:", entropy)
    return score_dict

def discrete_mutual_info(mus, ys):
    num_codes = mus.shape[0]
    num_factors = ys.shape[0]
    m = np.zeros([num_codes, num_factors])
    
    for i in range(num_codes):
        for j in range(num_factors):
            m[i, j] = mutual_info_score(ys[j, :], mus[i, :])
    
    return m

def discrete_entropy(ys):
    num_factors = ys.shape[0]
    h = np.zeros(num_factors)
    
    for j in range(num_factors):
        h[j] = mutual_info_score(ys[j, :], ys[j, :])
    
    return h

def _identity_discretizer(target, num_bins):
    del num_bins
    return target


def _histogram_discretize(target, num_bins=10):
    discretized = np.zeros_like(target)
    for i in range(target.shape[0]):
        discretized[i, :] = np.digitize(target[i, :], np.histogram(
            target[i, :], num_bins)[1][:-1])
    return discretized


def make_discretizer(target, num_bins=10, discretizer_fn=_histogram_discretize):
    return discretizer_fn(target, num_bins)


def score_disentanglement(adata, embedding_data, embedding_basal, covriate_keys=None, continuous_covriate_keys=None):
    mus, ys = prep_data(adata, embedding_data, covriate_keys=covriate_keys)
    print('Computing MIG')
    mig = compute_mig(mus, ys, covariate_names=covriate_keys)
    return mig, mus, ys

# Run MIG score
mig_1,mus,ys = score_disentanglement(
    adata,
    latent,
    None,
    covriate_keys=["cell_type", "sex", "donor_id"]
)

print("MIG Score:", mig_1)
print("Latent variances:", np.var(mus, axis=1))

discretized_mus = _histogram_discretize(mus)
m = discrete_mutual_info(discretized_mus, ys)

print("MI matrix shape:", m.shape)
print("Max MI per factor:", np.max(m, axis=0))
print("Which latents have highest MI per factor:", np.argmax(m, axis=0))




In [ ]:
# Finalized DCI computation based on disentanglement_lib

import numpy as np
import pandas as pd
import anndata
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import entropy

# === Encoding and Preprocessing ===
def encode_categorical(data):
    encoded_data = np.zeros_like(data, dtype=int)
    for i in range(data.shape[1]):
        le = LabelEncoder()
        encoded_data[:, i] = le.fit_transform(data[:, i])
    return encoded_data

def remove_duplicate_columns(df):
    df_unique = df.T.drop_duplicates().T
    return df_unique

def prep_data(adata, embedding, covariate_keys, test_size=0.25):
    idx_train, idx_test = train_test_split(
        range(len(adata)), test_size=test_size, random_state=42
    )
    cov_df = adata.obs[covariate_keys].copy()
    cov_df = remove_duplicate_columns(cov_df)
    encoded_factors = encode_categorical(cov_df.values)
    embedding_data = embedding.X if isinstance(embedding, anndata.AnnData) else embedding
    mus_train = embedding_data[idx_train]
    mus_test = embedding_data[idx_test]
    ys_train = encoded_factors[idx_train]
    ys_test = encoded_factors[idx_test]
    return mus_train.T, ys_train.T, mus_test.T, ys_test.T

# === Importance Matrix ===
def compute_importance_rf(x_train, y_train, x_test, y_test):
    num_factors = y_train.shape[0]
    num_codes = x_train.shape[0]
    importance_matrix = np.zeros((num_codes, num_factors))
    train_acc = []
    test_acc = []
    for i in range(num_factors):
        model = RandomForestClassifier(random_state=42, max_depth=5)
        model.fit(x_train.T, y_train[i])
        importance_matrix[:, i] = np.abs(model.feature_importances_)
        train_acc.append(np.mean(model.predict(x_train.T) == y_train[i]))
        test_acc.append(np.mean(model.predict(x_test.T) == y_test[i]))
    return importance_matrix, np.mean(train_acc), np.mean(test_acc)

# === Disentanglement ===
def disentanglement_per_code(importance_matrix):
    row_sums = importance_matrix.sum(axis=1, keepdims=True)
    safe_matrix = np.where(row_sums == 0, 1e-11, row_sums)
    normalized = importance_matrix / safe_matrix
    return 1. - entropy(normalized.T + 1e-11, base=importance_matrix.shape[1])

def disentanglement(importance_matrix):
    per_code = disentanglement_per_code(importance_matrix)
    total = importance_matrix.sum()
    if total == 0.:
        return 0.0
    code_importance = importance_matrix.sum(axis=1) / total
    return np.sum(per_code * code_importance)

# === Completeness ===
def completeness_per_factor(importance_matrix):
    return 1. - entropy(importance_matrix + 1e-11, base=importance_matrix.shape[0])

def completeness(importance_matrix):
    per_factor = completeness_per_factor(importance_matrix)
    total = importance_matrix.sum()
    if total == 0.:
        return 0.0
    factor_importance = importance_matrix.sum(axis=0) / total
    return np.sum(per_factor * factor_importance)

# === DCI Master Function ===
def compute_dci(mus_train, ys_train, mus_test, ys_test):
    importance_matrix, train_acc, test_acc = compute_importance_rf(
        mus_train, ys_train, mus_test, ys_test
    )
    threshold = 1e-11
    importance_matrix = np.where(importance_matrix < threshold, 0, importance_matrix)
    return {
    "disentanglement": disentanglement(importance_matrix),
    "completeness": completeness(importance_matrix),
    "importance_matrix": importance_matrix,
    "train_acc": train_acc,
    "test_acc": test_acc,
}


In [ ]:

covariate_keys = ["cell_type", "sex", "donor_id"]
mus_train, ys_train, mus_test, ys_test = prep_data(adata, latent, covariate_keys)
dci_scores = compute_dci(mus_train, ys_train, mus_test, ys_test)


print("Disentanglement Score:", dci_scores["disentanglement"])
print("Completeness Score:", dci_scores["completeness"])
print("Train Accuracy:", dci_scores["train_acc"])
print("Test Accuracy:", dci_scores["test_acc"])

# Access importance matrix if needed
importance_matrix = dci_scores["importance_matrix"]


In [ ]:
#SAP score
from sklearn import svm

def compute_sap(mus, ys, mus_test, ys_test, continuous_factors):
    """Computes the SAP score.

    Args:
        mus, ys, mus_test, ys_test
        continuous_factors: Factors are continuous variable (True) or not (False).

    Returns:
        Dictionary with SAP score.
    """

    return _compute_sap(mus, ys, mus_test, ys_test, continuous_factors)

def _compute_sap(mus, ys, mus_test, ys_test, continuous_factors):
    """Computes score based on both training and testing codes and factors."""
    score_matrix = compute_score_matrix(mus, ys, mus_test, ys_test, continuous_factors)
    # Score matrix should have shape [num_latents, num_factors].
    assert score_matrix.shape[0] == mus.shape[0]
    assert score_matrix.shape[1] == ys.shape[0]
    scores_dict = {}
    scores_dict["SAP_score"] = compute_avg_diff_top_two(score_matrix)

    return scores_dict

def compute_score_matrix(mus, ys, mus_test, ys_test, continuous_factors):
    """Compute score matrix as described in Section 3."""
    num_latents = mus.shape[0]
    num_factors = ys.shape[0]
    score_matrix = np.zeros([num_latents, num_factors])
    for i in range(num_latents):
        for j in range(num_factors):
            mu_i = mus[i, :]
            y_j = ys[j, :]
            if continuous_factors:
                # Attribute is considered continuous.
                cov_mu_i_y_j = np.cov(mu_i, y_j, ddof=1)
                cov_mu_y = cov_mu_i_y_j[0, 1]**2
                var_mu = cov_mu_i_y_j[0, 0]
                var_y = cov_mu_i_y_j[1, 1]
                if var_mu > 1e-12:
                    score_matrix[i, j] = cov_mu_y * 1. / (var_mu * var_y)
                else:
                    score_matrix[i, j] = 0.
            else:
                # Attribute is considered discrete.
                mu_i_test = mus_test[i, :]
                y_j_test = ys_test[j, :]
                classifier = svm.LinearSVC(C=0.01, class_weight="balanced")
                classifier.fit(mu_i[:, np.newaxis], y_j)
                pred = classifier.predict(mu_i_test[:, np.newaxis])
                score_matrix[i, j] = np.mean(pred == y_j_test)
    return score_matrix

def compute_avg_diff_top_two(matrix):
    sorted_matrix = np.sort(matrix, axis=0)
    return np.mean(sorted_matrix[-1, :] - sorted_matrix[-2, :])

sap = compute_sap(mus_train, ys_train, mus_test, ys_test, continuous_factors=False)
sap

In [ ]:
# IRS 


def compute_irs(mus, ys, diff_quantile=0.99):
    ys_discrete = make_discretizer(ys)

    active_mask = (mus.var(axis=1) > 0)
    active_mus = mus[active_mask, :]

    if active_mus.size == 0:
        irs_score = 0.0
    else:
        irs_score = scalable_disentanglement_score(ys_discrete.T, active_mus.T, diff_quantile)["avg_score"]

    score_dict = {}
    score_dict["IRS"] = irs_score
    score_dict["num_active_dims"] = int(np.sum(active_mask))
    return score_dict


def _drop_constant_dims(ys):
    """Returns a view of the matrix `ys` with dropped constant rows."""
    ys = np.asarray(ys)
    if ys.ndim != 2:
        raise ValueError("Expecting a matrix.")

    variances = ys.var(axis=1)
    active_mask = variances > 0.
    return ys[active_mask, :]


def scalable_disentanglement_score(gen_factors, latents, diff_quantile=0.99):
    """Computes IRS scores of a dataset.

    Assumes no noise in X and crossed generative factors (i.e. one sample per
    combination of gen_factors). Assumes each g_i is an equally probable
    realization of g_i and all g_i are independent.

    Args:
        gen_factors: Numpy array of shape (num samples, num generative factors),
            matrix of ground truth generative factors.
        latents: Numpy array of shape (num samples, num latent dimensions), matrix
            of latent variables.
        diff_quantile: Float value between 0 and 1 to decide what quantile of diffs
            to select (use 1.0 for the version in the paper).

    Returns:
        Dictionary with IRS scores.
    """
    num_gen = gen_factors.shape[1]
    num_lat = latents.shape[1]

    # Compute normalizer.
    max_deviations = np.max(np.abs(latents - latents.mean(axis=0)), axis=0)
    cum_deviations = np.zeros([num_lat, num_gen])
    for i in range(num_gen):
        unique_factors = np.unique(gen_factors[:, i], axis=0)
        assert unique_factors.ndim == 1
        num_distinct_factors = unique_factors.shape[0]
        for k in range(num_distinct_factors):
            # Compute E[Z | g_i].
            match = gen_factors[:, i] == unique_factors[k]
            e_loc = np.mean(latents[match, :], axis=0)

            # Difference of each value within that group of constant g_i to its mean.
            diffs = np.abs(latents[match, :] - e_loc)
            max_diffs = np.percentile(diffs, q=diff_quantile*100, axis=0)
            cum_deviations[:, i] += max_diffs
        cum_deviations[:, i] /= num_distinct_factors
    # Normalize value of each latent dimension with its maximal deviation.
    normalized_deviations = cum_deviations / max_deviations[:, np.newaxis]
    irs_matrix = 1.0 - normalized_deviations
    disentanglement_scores = irs_matrix.max(axis=1)
    if np.sum(max_deviations) > 0.0:
        avg_score = np.average(disentanglement_scores, weights=max_deviations)
    else:
        avg_score = np.mean(disentanglement_scores)

    parents = irs_matrix.argmax(axis=1)
    score_dict = {}
    score_dict["disentanglement_scores"] = disentanglement_scores
    score_dict["avg_score"] = avg_score
    score_dict["parents"] = parents
    score_dict["IRS_matrix"] = irs_matrix
    score_dict["max_deviations"] = max_deviations
    return score_dict



irs = compute_irs(mus, ys, diff_quantile=1)
print(irs)


print("mus shape:", mus.shape)


def compute_latent_sensitivity(mus, ys):
    num_latents = mus.shape[0]
    num_factors = ys.shape[0]
    sensitivity = np.zeros((num_latents, num_factors))

    for i in range(num_factors):  # for each generative factor
        factor_vals = np.unique(ys[i])
        group_means = []

        for val in factor_vals:
            idx = ys[i] == val
            mean_latents = np.mean(mus[:, idx], axis=1)  # shape: (num_latents,)
            group_means.append(mean_latents)

        group_means = np.stack(group_means, axis=1)  # shape: (num_latents, num_values)
        sensitivity[:, i] = np.std(group_means, axis=1)

    return sensitivity

import numpy as np

# Remove suspicious latents
mus_cleaned = np.delete(mus, suspicious_latents, axis=0)

irs_cleaned = compute_irs(mus_cleaned, ys, diff_quantile=0.99)
print("IRS after removing suspicious latents:", irs_cleaned)

sensitivity_matrix = compute_latent_sensitivity(mus, ys)  # shape: (latent_dim, factor_dim)
#print(sensitivity_matrix)

import matplotlib.pyplot as plt

latent_dim, factor_dim = sensitivity_matrix.shape
for i in range(factor_dim):
    plt.scatter(sensitivity_matrix[:, i], irs_matrix[:, i])
    plt.xlabel(f"Sensitivity to factor {i}")
    plt.ylabel(f"IRS for factor {i}")
    plt.title(f"IRS vs Sensitivity (Factor {i})")
    plt.grid(True)
    plt.show()



from sklearn.feature_selection import mutual_info_regression

def compute_mutual_info_matrix(mus, ys):
    num_latents = mus.shape[0]
    num_factors = ys.shape[0]
    mi_matrix = np.zeros((num_latents, num_factors))

    for j in range(num_latents):
        for i in range(num_factors):
            mi_matrix[j, i] = mutual_info_regression(ys[i:i+1].T, mus[j], discrete_features=True)[0]

    return mi_matrix
mi_matrix = compute_mutual_info_matrix(mus, ys)  # shape: (latent_dim, factor_dim)

import seaborn as sns

#sns.heatmap(mi_matrix, cmap='viridis')
#plt.title("Mutual Information between Latents and Factors")
#plt.xlabel("Factor Index")
#plt.ylabel("Latent Dimension")
#plt.show()

suspicious_latents = []
for j in range(mus.shape[0]):
    if np.all(sensitivity_matrix[j] < 0.01):  # low variation
        if np.sum(mi_matrix[j] > 0.01) > 1:    # entangled
            suspicious_latents.append(j)
print("Suspicious latents (stable + entangled):", suspicious_latents)


max_devs = irs_full["max_deviations"]
disent_scores = irs_full["disentanglement_scores"]

plt.scatter(max_devs, disent_scores)
plt.xlabel("Max Deviation (overall activity)")
plt.ylabel("IRS Score")
plt.title("Latent Activity vs IRS")
plt.grid(True)
plt.show()

